In [1]:
from __future__ import print_function # only needed in Python 2
import sys
from rx import Observable, Observer
from kafka import KafkaProducer, KafkaConsumer
import io
import avro.schema
import avro.io
# import local stuff
sys.path.append('../rx')
from KafkaObserver import KafkaObserver


In [2]:
# create avro serializer for our stream into byte arrays
# properly speaking, should be integrated into KafkaObserver
test_schema = '''
{
 "type": "record",
 "name": "Test",
 "fields": [ {"name": "value",  "type": ["int","null"]} ]
}
'''

schema = avro.schema.parse(test_schema)

def avroEncode(dict, schema):
    writer = avro.io.DatumWriter(schema)
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer.write(dict, encoder)
    return bytes_writer.getvalue()

def avroDecode(msg,schema):
    bytes_reader = io.BytesIO(msg)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

    
x=avroEncode({'value':1}, schema)
avroDecode(x, schema)

{u'value': 1}

In [3]:
topic='serializertest'
kobs=KafkaObserver(KafkaProducer(),topic)
obs=Observable.from_iterable(range(6,20,2)).map(lambda x: {'value':x})

In [4]:
obs.map(lambda x: avroEncode(x, schema)).subscribe(kobs) # successfully dumps byte-encoded records to Kafka

Sequence completed


In [8]:
from kafka import KafkaConsumer
consumer = KafkaConsumer(topic,group_id=None,auto_offset_reset='earliest')
#consumer.subscribe(topic=None, pattern="my-topic")
#consumer.subscription()

In [ ]:
# this one blocks while waiting for messages - how to transform it to a non-blocking version?
for msg in consumer:
    print(avroDecode(msg.value,schema))
  

{u'value': 8}
{u'value': 10}
{u'value': 12}
{u'value': 14}
{u'value': 16}
{u'value': 18}
